In [1]:
from AlgorithmImports import *
from System.Drawing import Color              

ModuleNotFoundError: No module named 'System'

In [4]:
class QC(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2017,1,1) #date start for backtest
        self.SetEndDate(2022, 8,8) #date end for backtest
        self.SetCash(100) #starting cash balance
        self.pair = self.AddForex("EURUSD", Resolution.Daily, Market.Oanda).Symbol #adding forex pair
        self.bb = self.BB(self.pair, 20, 2) #bollinger band over 20 days within 2 standard deviations
        
        Plotstock = Chart("Trade Plot") #Creates a plot
        Plotstock.AddSeries(Series("Buy", SeriesType.Scatter, "$", Color.Green, ScatterMarkerSymbol.Triangle)) #Plot of Buy
        Plotstock.AddSeries(Series("Sell", SeriesType.Scatter, "$", Color.Red, ScatterMarkerSymbol.TriangleDown)) #Plot of Sell
        Plotstock.AddSeries(Series("Liquidate", SeriesType.Scatter, "$", Color.Blue, ScatterMarkerSymbol.Diamond)) #Plot of Liquidation
        self.AddChart(Plotstock) #Adds the chart to the algorithm
    def OnData(self, data):
        if not self.bb.IsReady:
            return
        price = data[self.pair].Price
        
        self.Plot("Trade Plot", "Price", price) #Plot price
        self.Plot("Trade Plot", "MiddleBand", self.bb.MiddleBand.Current.Value) #Plots average
        self.Plot("Trade Plot", "Price", self.bb.UpperBand.Current.Value) #Plots long position
        self.Plot("Trade Plot", "Price", self.bb.LowerBand.Current.Value) #Plots short position
        
        if not self.Portfolio.Invested: #if not invested, check if price is below bollinger bands
            if self.bb.LowerBand.Current.Value > price:
                self.SetHoldings(self.pair,1) #buy as much of pair as possible
                self.Plot("Trade Plot", "Buy", price) #Marks on the plot entry position
            elif self.bb.UpperBand.Current.Value < price: #If above 2 s.d. establish short position
                self.SetHoldings(self.pair, -1)
                self.Plot("Trade Plot", "Sell", price) #Marks on the plot the exit position
            else:
                if self.Portfolio[self.pair].IsLong: #if in long position, liquidate the position
                    if self.bb.MiddleBand.Current.Value < price:
                        self.Liquidate()
                        self.Plot("Trade Plot", "Liquidate", price) #Marks on the plot the liquidate position
                elif self.bb.MiddleBand.Curent.Value > price: #if in short position, check if price dropped before MA and close any open positions
                    self.Liquidate()
                    self.Plot("Trade Plot", "Liquidate", price) #Marks on the plot the liquidate position
                

NameError: name 'QCAlgorithm' is not defined